In [19]:
import keras.models
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers

In [20]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [21]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Data import

In [22]:
dataset_dir = './SplitData/data'
training_dir = './SplitData/data/training'
validation_dir = './SplitData/data/validation'
test_dir = './SplitData/data/test'

In [23]:
# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator(rescale=1. / 255)
valid_data_gen = ImageDataGenerator(rescale=1. / 255)
test_data_gen = ImageDataGenerator(rescale=1. / 255)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(96, 96),
                                               color_mode='rgb',
                                               classes=None,  # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(96, 96),
                                               color_mode='rgb',
                                               classes=None,  # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed)
test_gen = train_data_gen.flow_from_directory(directory=test_dir,
                                              target_size=(96, 96),
                                              color_mode='rgb',
                                              classes=None,  # can be set to labels
                                              class_mode='categorical',
                                              batch_size=32,
                                              shuffle=False,
                                              seed=seed)

Found 2835 images belonging to 8 classes.
Found 398 images belonging to 8 classes.
Found 495 images belonging to 8 classes.


## Augmentation

In [24]:
tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
    height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=False, vertical_flip=False, rescale=None,
    preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
)

In [25]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=20,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        brightness_range=(0.6, 1.4),
                                        zoom_range=0.6,
                                        fill_mode='nearest',
                                        rescale=1. / 255)  # rescale value is multiplied to the image

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=training_dir,
                                                       target_size=(96, 96),
                                                       color_mode='rgb',
                                                       classes=None,  # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       seed=seed)

Found 2835 images belonging to 8 classes.


## Model

In [26]:
input_shape = (96, 96, 3)
epochs = 200

In [44]:
from sklearn.utils import class_weight

class_weights = {0: 2., 1: 1., 2: 1., 3: 1., 4: 1., 5: 2., 6: 1., 7: 1.}

In [34]:
def build_model(input_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    conv1 = tfkl.Conv2D(
        filters=32,
        kernel_size=3,
        padding='same',
        activation='relu',
        kernel_initializer=tfk.initializers.HeUniform(seed)
    )(input_layer)
    pool1 = tfkl.MaxPooling2D()(conv1)

    conv2 = tfkl.Conv2D(
        filters=64,
        kernel_size=3,
        padding='same',
        activation='relu',
        kernel_initializer=tfk.initializers.HeUniform(seed)
    )(pool1)
    pool2 = tfkl.MaxPooling2D()(conv2)

    conv3 = tfkl.Conv2D(
        filters=128,
        kernel_size=3,
        padding='same',
        activation='relu',
        kernel_initializer=tfk.initializers.HeUniform(seed)
    )(pool2)
    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=3,
        padding='same',
        activation='relu',
        kernel_initializer=tfk.initializers.HeUniform(seed)
    )(conv3)
    pool3 = tfkl.MaxPooling2D()(conv4)

    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=3,
        padding='same',
        activation='relu',
        kernel_initializer=tfk.initializers.HeUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D()(conv5)

    conv6 = tfkl.Conv2D(
        filters=256,
        kernel_size=3,
        padding='same',
        activation='relu',
        kernel_initializer=tfk.initializers.HeUniform(seed)
    )(pool4)
    pool5 = tfkl.MaxPooling2D()(conv6)

    flattening_layer = tfkl.GlobalAvgPool2D(name='gap')(pool5)
    dropout = tfkl.Dropout(0.2, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=256, name='Classifier', kernel_initializer=tfk.initializers.HeUniform(seed),
                                  activation='relu')(dropout)
    dropout = tfkl.Dropout(0.2, seed=seed)(classifier_layer)
    output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed),
                              name='output_layer')(dropout)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [28]:
from datetime import datetime


def create_folders_and_callbacks(model_name):
    exps_dir = os.path.join('data_augmentation_experiments')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)

    callbacks = []

    # Model checkpoint
    # ----------------
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'),
                                                       save_weights_only=True,  # True to save only weights
                                                       save_best_only=False)  # True to save only the best epoch
    callbacks.append(ckpt_callback)

    # Visualize Learning on Tensorboard
    # ---------------------------------
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)

    # By default shows losses and metrics for both training and validation
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                                 profile_batch=0,
                                                 histogram_freq=1)  # if > 0 (epochs) shows weights histograms
    callbacks.append(tb_callback)

    # Early Stopping
    # --------------
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    callbacks.append(es_callback)

    return callbacks

In [45]:
# Build model (for data augmentation training)
model = build_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 96, 96, 32)        896       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 48, 48, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 48, 48, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 24, 24, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 24, 24, 128)       73856 

In [46]:
# Create folders and callbacks and fit
aug_callbacks = create_folders_and_callbacks(model_name='CNN_Aug')

# Train the model
history = model.fit(
    x=aug_train_gen,
    epochs=epochs,
    validation_data=valid_gen,
    callbacks=aug_callbacks,
    batch_size=32,
    class_weight=class_weights
).history

Epoch 1/200
89/89 [==============================] - 11s 111ms/step - loss: 2.9943 - accuracy: 0.1072 - val_loss: 2.0767 - val_accuracy: 0.1206
Epoch 2/200
89/89 [==============================] - 9s 103ms/step - loss: 2.8487 - accuracy: 0.1566 - val_loss: 2.0197 - val_accuracy: 0.2563
Epoch 3/200
89/89 [==============================] - 8s 94ms/step - loss: 2.7230 - accuracy: 0.2120 - val_loss: 1.7996 - val_accuracy: 0.2563
Epoch 4/200
89/89 [==============================] - 8s 90ms/step - loss: 2.7248 - accuracy: 0.2049 - val_loss: 2.0683 - val_accuracy: 0.2487
Epoch 5/200
89/89 [==============================] - 8s 94ms/step - loss: 2.6184 - accuracy: 0.2705 - val_loss: 1.9145 - val_accuracy: 0.2889
Epoch 6/200
89/89 [==============================] - 8s 87ms/step - loss: 2.5204 - accuracy: 0.3323 - val_loss: 1.7328 - val_accuracy: 0.3593
Epoch 7/200
89/89 [==============================] - 8s 88ms/step - loss: 2.4367 - accuracy: 0.3386 - val_loss: 1.7246 - val_accuracy: 0.3518
Epo

In [47]:
# Save best epoch model
model.save("data_augmentation_experiments/CNN_Aug_Best")

In [48]:
# Trained with data augmentation
model_aug = tfk.models.load_model("data_augmentation_experiments/CNN_Aug_Best")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with data augmentation")
print(model_aug_test_metrics)


16/16 [==============================] - 1s 30ms/step - loss: 0.8971 - accuracy: 0.6828

Test metrics with data augmentation
{'loss': 0.8970720171928406, 'accuracy': 0.6828283071517944}
